This notebook looks at the output from the snippet repository
and how to use it to train NER, classification, and mlm models.

In [1]:

from typing import List
from spacy import displacy
import src.data.snippet_repository as sr

### Named Enitiy Recognition Models (NER)

In [3]:
ner_repo = sr.SnippetRepository(sr.SnippetRepositoryMode.NER)

In [131]:
ner_data = ner_repo.get_training_data(batch_size=10)
detected = False
while not detected:
    ner_df = next(ner_data)
    detected = any(ner_df['ner_tags'].apply(lambda ner_tags: any(map(lambda t: t!="O", ner_tags))))

In [146]:
text = ner_df.iloc[9].text
ner_tags = ner_df.iloc[9].ner_tags

In [147]:
def extract_entities(text:List[str], ner_tags:List[str]):
    entities = []
    start = 0
    end = 0
    in_ent = False
    for token, tag in zip(text, ner_tags):
        token_length = len(token) + 1 # plus a space
        if tag == "I-DAT":
            in_ent = True
            end += token_length
        elif tag == "O" and in_ent:
            entities.append({
                "start": start,
                "end": end,
                "label": "Dataset"
            })
            in_ent = False
            start = end
        elif tag == "B-DAT":
            entities.append({
                "start": start,
                "end": end,
                "label": "Dataset"
            })
            start = end
            end += token_length
        else:
            # just an O tag
            start += token_length
            end = start

    return entities

entities = extract_entities(text, ner_tags)
ex = [{
    "text": " ".join(text),
    "ents": entities,
    "title": None 
}]

displacy.render(ex, style="ent", manual=True, options={"colors":{"Dataset": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}})

In [ ]:
# load the model here